Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [0]:
# importing numpy and scipy stats
from scipy.stats import mode
import numpy as np


In [69]:
train.head(1)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional


In [75]:
train.shape

(59400, 41)

In [72]:
target = 'status_group'
y_train = train[target]
y_train.shape

(59400,)

In [74]:
x_train = train.drop('status_group', axis=1)
x_train.shape

(59400, 40)

In [0]:

# Creating the list of the columns that I want to drop here and
# then will pass them into the wrangle function
target = 'status_group'
# This is a list of some of the colums that I want to drop
# Will drop subvillage -- because similar results as lat and lon
# Dropping funder:  has high cardinality
# Dropping the wpt_name:  has high cardinality is the name of the water point
# Dropping payment_type:  same as payment(duplicate)
cols_to_drop = ['id', 'quantity_group', 'recorded_by', "subvillage", 
                'num_private', 'funder', 'wpt_name' ,'scheme_name', 'payment_type',
                'quality_group', ]

In [0]:
# These are some inner functions in my wrangle function that help
# With the fixing the latitude and the longitude
def myLatitude(theRow,df):
  
  if pd.isnull(theRow['latitude']):
    theWard = theRow['ward']
    lat = df[df['ward'] == theWard]
    theMode = mode(lat['latitude'])
    return (theMode[0][0])
  return (theRow['latitude'])

def myLon(theRow, df): 
  if (pd.isnull(theRow['longitude'])):
    lon = df[df['ward'] == theRow['ward']]
    theMode = mode(lon['longitude'])
    return (theMode[0][0])
  return (theRow['longitude'])

In [0]:
# This is my wrangle function

def wrangleFxn(df, cols_to_drop ,numCardinal=50, useMode=True, num=None, highVar=None, lowVar=None, useDropList=True ):
  ''' This is the wrangle function
      It will take in a dataFrame either the train, test, or validate.
      It should have the TARGET already removed from it. 

      cols_to_drop:   List of features that you want to drop automatically from 
                      the dataFrame
                      It will drop the columns before looking at the cardinality of the
                      columns.

      numCardinal:  default is 50. This is the threshold.  If it is less
                    than  or equal to this number it is retained in the 
                    features.
      useMode:  default is True.  This will use the apply functiontion to 
                make the lat and lon have values similar to those in 
                the ward that it is found.
      num:      default is None.  It this is set to a number, 
                it will further reduce the cardinality of the
                cardinal features to a specified number. 
      
      highVar:  Default is None. If set will drop any columns that have more 
                variance than what is set here as the threshold.  Should be
                set as a float ie(.90 or .8)
      
      lowVar:   Default is None. if set will drop any columns who have less
                variance than here.  Should be set as a float ie(.1 or .234)

      useDropList:  Default is True.  When is True will use the cols_to_drop parameter.
                    When false will not use the cols_to_drop parameter 

      returns:  Will return the datframe prepared for the pipeline.

  '''
  
  
  target = 'status_group'
  # making the copy
  df = df.copy()
  # will try to drop the target if it is present
  if target in df.columns:
    df = df.drop(columns=[target])
  
  # Will be setting the long and lat before doing any dropping
  df['latitude'] = df['latitude'].replace(-2e-08, 0)

  # Getting rid of the zeros in the latitude and the longitude putting in 
  # np.nan
  cols_w_zeros = ['latitude', 'longitude']

  for col in cols_w_zeros:
    df[col] = df[col].replace(0, np.nan)

  if useMode == True:
    #This is to make the latitude and the longitude have
    # numbers similar to those whose region they are in
    df['latitude'] = df.apply(myLatitude, axis=1, args=(df,))
    df['longitude'] = df.apply(myLon, axis=1, args=(df,))
    pd.to_numeric(df['latitude'])
  
  # Changing the format to dateTime format and then
  # making 
  df['date_recorded'] = pd.to_datetime(df['date_recorded'])
  df['construction_year'] = pd.to_datetime(df['construction_year'])
  df['construction_year'] = df['construction_year'].dt.year

  df['year_recorded'] = df['date_recorded'].dt.year
  df['month_recorded'] = df['date_recorded'].dt.month
  df['day_recorded'] = df['date_recorded'].dt.day

  # Making a new feature that will be "year_age"
  df['year_age']  = df['year_recorded'] - df['construction_year']
  
  # Dropping the columns preset to drop
  if useDropList == True:
    df = df.drop(columns= cols_to_drop)

  # Getting a list of the cardinal features at the start
  cardinals = df.select_dtypes(exclude='number')

  # This to drop features that are left that have high skew (almost no variance)
  # or possibly too much variance
  if highVar != None:
    # Getting a list of the cardinal features at the start
    cardinals = df.select_dtypes(exclude='number')
    tCol = cardinals.columns.to_list()
    for col in tCol:
      if df[col].value_counts(normalize=True, dropna=False).to_list()[0]  > highVar:
        df = df.drop(columns=[col])
  
  if lowVar != None:
    # Getting a list of the cardinal features at the start
    cardinals = df.select_dtypes(exclude='number')
    tCol = cardinals.columns.to_list()
    
    for col in tCol:
      if df[col].value_counts(normalize=True, dropna=False).to_list()[0] < lowVar:
        df = df.drop(columns=[col])


  # Getting the number of features that are numerical
  numerical_features = df.select_dtypes(include='number').columns.tolist()

  # Getting the amounts in each feature of the cardinal features
  cardinalFeatureAmounts = df.select_dtypes(exclude='number')
  cardinalFeatureAmounts = cardinalFeatureAmounts.nunique()
  # checking to see if 'ward' is in the list

  # Creating a list of the features that have less than 50 cardinality
  cardinal = cardinalFeatureAmounts[cardinalFeatureAmounts <= numCardinal].index.tolist()
  
  if num != None:
    #This is to reduce the number of features in the cardinal groups
    for feature in cardinal:
      # getting the top specified amount
      topNum = df[feature].value_counts().index[:num-1].tolist()
      # replace those that are not in there with 'other'
      df.loc[~df[feature].isin(topNum), feature] = 'Other'

  # List of all the features
  features = numerical_features + cardinal

  
  

  return df[features]

In [0]:
x_train1 = wrangleFxn(x_train, cols_to_drop=cols_to_drop, num=None, highVar=.90, lowVar=.06, useDropList=True)

x_test1 = wrangleFxn(test, cols_to_drop=cols_to_drop, num=None, highVar=.90, lowVar=.06, useDropList=True)

In [81]:
x_train1.columns

Index(['amount_tsh', 'gps_height', 'longitude', 'latitude', 'region_code',
       'district_code', 'population', 'construction_year', 'year_recorded',
       'month_recorded', 'day_recorded', 'year_age', 'basin', 'region',
       'public_meeting', 'scheme_management', 'permit', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'water_quality', 'quantity', 'source',
       'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group'],
      dtype='object')

In [0]:
# Doing some imports for sklearn
from  sklearn.metrics import accuracy_score
from  sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import category_encoders as ce 
from sklearn.model_selection import RandomizedSearchCV


In [0]:
# creating the pipeline
pipeline = make_pipeline(ce.OrdinalEncoder(),
                         SimpleImputer(), 
                         RandomForestClassifier(random_state=42))

In [0]:
# getting the params for the RandomizedSearch
params = {
    
    'simpleimputer__strategy': ['mean', 'median'],
    'randomforestclassifier__n_estimators': [100, 150, 175, 200],
    'randomforestclassifier__max_depth': [5,10, 15,20],
    'randomforestclassifier__min_samples_leaf': [1,2,3],
    'randomforestclassifier__min_samples_split': [2,3,4]
}

In [0]:
# Trying to run the crossvalidation
search = RandomizedSearchCV(pipeline, param_distributions=params, 
                            n_iter=5, n_jobs=4, random_state=42, 
                            return_train_score=True, verbose=10)

In [110]:
search.fit(x_train1, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   49.1s
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  1.8min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  2.6min
[Parallel(n_jobs=4)]: Done  21 out of  25 | elapsed:  3.4min remaining:   38.9s
[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:  3.8min finished


RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            fill_value=Non

In [111]:
print('Best hyperparameters are', search.best_params_)
print("Cross_val MAE", -search.best_score_)

Best hyperparameters are {'simpleimputer__strategy': 'mean', 'randomforestclassifier__n_estimators': 100, 'randomforestclassifier__min_samples_split': 4, 'randomforestclassifier__min_samples_leaf': 2, 'randomforestclassifier__max_depth': 20}
Cross_val MAE -0.8072895622895622


In [0]:
thePipe = make_pipeline(ce.TargetEncoder(smoothing=1, min_samples_leaf=800), 
                        SimpleImputer(strategy='median'),
                        RandomForestClassifier(n_estimators=175, min_samples_split=2,
                                               min_samples_leaf=3,
                                               max_depth=5))

In [0]:
thePipe.fit(x_train1, y_train)